# Nettoyage des données

## 0. Set-up

### Mise en place de l'environnement

In [1]:
import pandas as pd
import numpy as np

import sqlalchemy
import mysql.connector

import sys
sys.path.insert(0, "/home/apprenant/simplon_project/food_facts")


#On peut ensuite importer le dossier comme un module

from src.d00_utils.mysql_utils import mysql_connect, save_to_mysql
connect = mysql_connect()

In [2]:
food_info = pd.read_csv("/home/apprenant/simplon_project/food_facts/data/foodfacts.tsv", sep= '\t' , low_memory= False)

In [31]:
# on limite les données à 10000 lignes elle sont choisit aléatoirement 
df = food_info.sample(10000) 


In [32]:
df.columns.values

array(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name',
       'generic_name', 'quantity', 'packaging', 'packaging_tags',
       'brands', 'brands_tags', 'categories', 'categories_tags',
       'categories_en', 'origins', 'origins_tags', 'manufacturing_places',
       'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en',
       'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo',
       'cities', 'cities_tags', 'purchase_places', 'stores', 'countries',
       'countries_tags', 'countries_en', 'ingredients_text', 'allergens',
       'allergens_en', 'traces', 'traces_tags', 'traces_en',
       'serving_size', 'no_nutriments', 'additives_n', 'additives',
       'additives_tags', 'additives_en', 'ingredients_from_palm_oil_n',
       'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags',
       'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_that_may_be_from_palm_oil',
   

## 1. Sélection des colonnes

J'affiche un échantillon de mes données pour analyser la pertinence des différentes colonnes

In [33]:
print(df.head())
print(df.columns)

                 code                                                url  \
153064  0815369010740  http://world-en.openfoodfacts.org/product/0815...   
161898  0852793005218  http://world-en.openfoodfacts.org/product/0852...   
308848  5600317476335  http://world-en.openfoodfacts.org/product/5600...   
5717    0011150165716  http://world-en.openfoodfacts.org/product/0011...   
5923    0011150270274  http://world-en.openfoodfacts.org/product/0011...   

                creator   created_t      created_datetime last_modified_t  \
153064  usda-ndb-import  1489144126  2017-03-10T11:08:46Z      1489144126   
161898  usda-ndb-import  1489073062  2017-03-09T15:24:22Z      1489073063   
308848          kiliweb  1496003362  2017-05-28T20:29:22Z      1496004012   
5717    usda-ndb-import  1489096598  2017-03-09T21:56:38Z      1489096598   
5923    usda-ndb-import  1489053816  2017-03-09T10:03:36Z      1489053816   

       last_modified_datetime                                 product_name  \
15

Je choisis uniquement les colonnes qui pourraient m'être utiles pour mon étude


## 2. Valeurs manquantes

In [35]:
df = df[['product_name','countries','ingredients_text','ingredients_from_palm_oil_n','nutrition_grade_fr','energy_100g','fat_100g','sugars_100g','fiber_100g','proteins_100g','salt_100g','nutrition-score-fr_100g', 'sodium_100g', 'carbohydrates_100g','saturated-fat_100g']]

In [36]:
print(df.isnull().sum()) 
print(df.shape)

product_name                    509
countries                        10
ingredients_text               2014
ingredients_from_palm_oil_n    2014
nutrition_grade_fr             2817
energy_100g                    1700
fat_100g                       2121
sugars_100g                    2174
fiber_100g                     3835
proteins_100g                  1738
salt_100g                      1867
nutrition-score-fr_100g        2817
sodium_100g                    1869
carbohydrates_100g             2134
saturated-fat_100g             2588
dtype: int64
(10000, 15)


il me manque entre 1700 et 3900 données dans chaque colonnes sauf pour les colonnes contries , states et product_name ou il y a peu ou tres peu de valeurs manquantes

### Traitement de product_name

on supprime les lignes ou la valeur de la colonne product_name est nulle car le but de ses données nettoyer est de les utiliser pour une application donc il faut que le nom du produit apparait

In [37]:
df.dropna(subset = ['product_name'], inplace = True)

### Traitement de ingredients_text

In [38]:
print(df.ingredients_text.unique())
print(df.ingredients_text.value_counts())

['Carbonated water, organic sugar, organic apple and organic pomegranate juice from concentrate, citric acid, natural flavor, organic black carrot juice from concentrate.'
 'Carbonated water, sugar (cane), natural flavors, apple juice concentrate, citric acid, ascorbic acid, reb a (stevia extract), beta carotene (color).'
 nan ...
 'Sucre, beurre de cacao, poudre de _lait_ écrémé, pâte de cacao, lactosérum en poudre (de _lait_), _beurre_ concentré, pâte de _noisettes_, émulsifiant (lécithine de _soja_), arôme.'
 "Farine de blé, beurre pâtissier 26,7% (bêta-carotène), sucre, oeufs et jaunes d'oeufs frais, lait écrémé en poudre, carbonate acide de sodium, carbonate acide d'ammonium, acide tartrique, sel."
 "Farine de blé - beurre 30% - sucre - oeufs - sel - poudre à lever : carbonates d'ammonium - colorant : caroténoïdes. Traces de soja et de graines de sésame. Les informations en gras sont destinées intolérantes ou personnes aux allergiques."]
Carbonated water, natural flavor.          

on ne remplacera pas les valeurs manquantes , on ne peut pas donner la liste des elements du produit

In [39]:
print(df.proteins_100g.unique())
print(df.proteins_100g.value_counts())

[0.000e+00 1.640e+01 3.000e-01 1.200e+00 1.500e+00 1.667e+01 8.400e+00
 9.900e+00       nan 3.330e+00 5.650e+00 4.000e+00 3.200e+00 7.140e+00
 7.410e+00 6.000e-01 6.429e+01 6.200e+00 3.000e+01 8.700e+00 2.357e+01
 5.900e+00 3.570e+00 4.000e-01 2.100e+01 6.670e+00 8.110e+00 2.700e+00
 4.100e+00 4.200e+00 1.935e+01 1.000e+01 5.800e+00 1.250e+01 9.500e+00
 3.500e+00 3.850e+00 2.143e+01 6.700e+00 1.400e+01 7.500e+00 1.180e+00
 1.090e+01 1.600e+01 2.470e+00 2.080e+00 7.070e+00 4.400e+00 9.000e-01
 4.620e+00 1.300e+01 8.000e+00 5.000e-01 1.800e+00 8.300e-01 5.500e+00
 1.000e-01 9.550e+00 2.100e+00 9.000e+00 1.304e+01 1.120e+00 3.700e+00
 4.230e+00 1.000e+00 2.350e+00 6.780e+00 6.000e+00 1.333e+01 1.176e+01
 2.400e+00 7.300e+00 3.880e+00 4.700e+00 2.321e+01 1.429e+01 9.300e+00
 4.880e+00 7.000e-01 1.600e+00 1.400e+00 2.500e+01 6.300e+00 8.300e+00
 2.300e+01 2.000e+01 8.800e+00 8.970e+00 4.300e+00 4.500e+00 6.400e+00
 1.786e+01 7.700e+00 8.240e+00 8.500e+00 5.000e+00 1.857e+01 4.600e+00
 4.170

on remplace les valeurs 'nan' par zéro dans les colonnes séléctionnées 

In [40]:
list = ['ingredients_from_palm_oil_n','fat_100g','sugars_100g','fiber_100g','proteins_100g','salt_100g','sodium_100g', 'carbohydrates_100g','saturated-fat_100g']
def nan_to_value(df, list , v) : 
    """function to replace nan values by another one in several columns"""     
    d = {value:v for value in list}
    df.fillna(d , inplace = True)
    return df
nan_to_value(df, list , 0)


,product_name,countries,ingredients_text,ingredients_from_palm_oil_n,nutrition_grade_fr,energy_100g,fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,nutrition-score-fr_100g,sodium_100g,carbohydrates_100g,saturated-fat_100g
153064,"Organic Sparkling Italian Soda, Pomegranate",US,"Carbonated water, organic sugar, organic apple...",0.0,b,209.0,0.00,12.08,0.0,0.00,0.02540,2.0,0.010000,12.08,0.00
161898,Sparkling Beverage,US,"Carbonated water, sugar (cane), natural flavor...",0.0,NaN,71.0,0.00,4.06,0.0,0.00,0.00000,NaN,0.000000,4.23,0.00
308848,Graines de Chia Bio,France,NaN,0.0,a,1734.0,30.10,0.50,41.0,16.40,0.01000,-2.0,0.003937,0.90,3.70
5717,"Sparkling Water, Strawberry, Lemonade",US,"Carbonated water, citric acid, natural flavor,...",0.0,NaN,0.0,0.00,0.00,0.0,0.00,0.00508,NaN,0.002000,0.00,0.00
5923,Peanut Oil,US,Peanut oil.,0.0,NaN,3586.0,100.00,0.00,0.0,0.00,0.00000,NaN,0.000000,0.00,17.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170763,"Dark Chocolate, Orange",US,"Cocoa beans*+, cane sugar*+, cocoa butter*+, g...",0.0,d,2092.0,38.10,30.95,9.5,7.14,0.00000,17.0,0.000000,47.62,23.81
81885,"Go Lite!, Popcorn, Himalayan Salt",US,"Select popcorn, coconut oil, himalayan salt.",0.0,d,1891.0,19.77,0.00,11.3,11.30,1.43510,16.0,0.565000,67.80,14.12
328856,Vache Joyeuses Pâques,France,"Sucre, beurre de cacao, poudre de _lait_ écrém...",0.0,e,2220.0,29.50,58.00,1.8,6.30,0.43000,25.0,0.169291,59.00,17.50
213431,Palets breton pur beurre,France,"Farine de blé, beurre pâtissier 26,7% (bêta-ca...",0.0,e,2160.0,27.00,23.00,0.0,5.20,0.34000,22.0,0.133858,64.00,18.00


### Conclusion sur les valeurs manquantes 

J'ai gardé une grande partie des données il reste encore des valeurs manquantes mais celle ci peuvent etre remplit , on s'occupera de ca un peu plus tard

## 3. Traitement des dates

In [41]:
print(df.dtypes)


product_name                    object
countries                       object
ingredients_text                object
ingredients_from_palm_oil_n    float64
nutrition_grade_fr              object
energy_100g                    float64
fat_100g                       float64
sugars_100g                    float64
fiber_100g                     float64
proteins_100g                  float64
salt_100g                      float64
nutrition-score-fr_100g        float64
sodium_100g                    float64
carbohydrates_100g             float64
saturated-fat_100g             float64
dtype: object


Aucune de mes colonnes ne correspond à une date, je n'ai donc pas à faire de traitement

## 4. Traitement des doublons 

In [42]:
print(df.duplicated().value_counts())

False    9479
True       12
dtype: int64


on peut voir qu'il y a des doublons on va les supprimes

In [43]:
df = df.drop_duplicates(keep= 'first')

on relance la ligne pour verifier que les doublons sont supprimés

In [44]:
print(df.duplicated().value_counts())

False    9479
dtype: int64


Les doulons on bien été supprimés

## 5. Consistance des données 

on verifie que les valeurs soit corects 

In [45]:
def inconsistant_value(dataframe , column):
    """function to check if values in data are inconsistant """
    df = dataframe[column]
    for index,value in enumerate(df) :
        if value < 0 or value > 100 :
            df.drop(index)

def nutri_score(dataframe, column) : 
    """function to check if value in column nutrition_score are isconsistant """
    df = dataframe[column]
    for value in df :
        if value < -15 or value > 40 :
            print(value)

def delete_incosistant_value(dataframe , column):
    """function who delete line where values are inconsistant """
    df = dataframe[column]
    for value in df : 
        if value < 0 or value > 100 :
            df.drop()

verifions les valeurs de chaque colonnes 

In [47]:
inconsistant_value(df,'ingredients_from_palm_oil_n')

pas de valeur incoherante dans la colonne ingredients_from_palm_oil_n

pas de valeur incoherante dans la colonne ingredients_that_may_be_from_palm_oil_n

In [49]:
#delete_incosistant_value(food_info,'fat_100g' )
inconsistant_value(df, 'fat_100g')

pas de valeur incoherante dans la colonne fat_100g

In [50]:
inconsistant_value(df, 'sugars_100g')

pas de valeur incoherante dans la colonne sugar_100g

In [51]:
inconsistant_value(df, 'fiber_100g')

dans la colonne fiber_100g on a une valeur incoherente donc on va supprimer la ligne

In [52]:
inconsistant_value(df, 'proteins_100g')

pas de valeur incoherante dans la colonne sugar_100g

In [23]:
nutri_score(df, 'nutrition-score-fr_100g')

pas de valeur incohérente dans la colonne nutrition_score-fr_100g

On supprime les lignes ou la colonne prdduct name n'as pas de nom

on sauvegarde les données sur la base de données

In [61]:
save_to_mysql(db_connect=connect,df_to_save=food_info,df_name='df_clean')

MySQLInterfaceError: MySQL server has gone away

## 6. Valeurs Aberantes 

In [54]:
df['nutriment_sum'] = df['fat_100g'] + df['carbohydrates_100g'] + df['fiber_100g'] + df['proteins_100g'] + df['salt_100g'] 
df = df.drop(df[(df.nutriment_sum > 100) | (df.nutriment_sum < 70)].index)
df = df.drop(df[(df.sugars_100g > 100) | (df.sugars_100g < 0)].index)
df = df.drop(df[(df.fat_100g > 100) | (df.fat_100g < 0)].index)
df = df.drop(df[(df.proteins_100g > 100) | (df.proteins_100g < 0)].index)
df = df.drop(df[(df.salt_100g > 100) | (df.salt_100g < 0)].index)
df = df.drop(df[(df.fiber_100g > 100) | (df.fiber_100g < 0)].index)
df = df.drop(df[(df.energy_100g > 4000) | (df.energy_100g < 0)].index)
df.shape

(2537, 16)

## 7. recalcul du nutri-score


In [55]:
def nutri_score_value(value):
        if value < 0 :
            return 'a'
        elif value < 3 :
            return 'b'
        elif value < 11 :
            return 'c'
        elif value < 19 :
            return 'd'
        else:
            return 'e'

df['nutrition_grade_fr'] = df['nutrition-score-fr_100g'].apply(nutri_score_value)

In [30]:
df.groupby(['nutrition-score-fr_100g','nutrition_grade_fr']).size().sort_index()

nutrition-score-fr_100g  nutrition_grade_fr
-12.0                    a                        1
-10.0                    a                        1
-9.0                     a                        1
-8.0                     a                        2
-7.0                     a                        5
-6.0                     a                      109
-5.0                     a                       54
-4.0                     a                       41
-3.0                     a                       27
-2.0                     a                       39
-1.0                     a                       78
 0.0                     b                     1721
 1.0                     b                       27
 2.0                     b                       40
 3.0                     c                       35
 4.0                     c                       18
 5.0                     c                       22
 6.0                     c                       41
 7.0                

In [56]:
df.to_csv('/home/apprenant/simplon_project/food_facts/data/df_clean.csv' , index = False , sep = ',' , encoding = 'utf-8', line_terminator = '\n' )